In [ ]:
import json
import pandas as pd
import numpy as np
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf
simpler_sf.simple_salesforce()
from sqlalchemy import create_engine

In [ ]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
#ruts = "F:/martin/MetLife SFTP/ORIENTA_02022024.xlsx"
#rutsMetlife = pd.read_excel(ruts)


In [ ]:
rutsMetlife.head()


In [ ]:
rutsMetlife = rutsMetlife[
    [
        "POLIZA",
        "RUTASEGURADO",
        "DV",
        "RUTBENEFICIARIO",
        "RUTBENEFIARIO_DV",
        "RELACION",
    ]
]
rutsMetlife.shape

In [ ]:
rutsMetlife["Poliza"] = rutsMetlife["POLIZA"].astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("-", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace("–", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].astype(str)
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(".", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.replace(" ", "")
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.upper()
rutsMetlife["Poliza"] = rutsMetlife["Poliza"].str.strip()

rutsMetlife["Rut_Titular"] = rutsMetlife["RUTASEGURADO"].astype(str) + rutsMetlife["DV"].astype(str)
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("-", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace("–", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].astype(str)
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(".", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.replace(" ", "")
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.upper()
rutsMetlife["Rut_Titular"] = rutsMetlife["Rut_Titular"].str.strip()

rutsMetlife["Rut_Beneficiario"] = rutsMetlife["RUTBENEFICIARIO"].astype(str) + rutsMetlife["RUTBENEFIARIO_DV"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("-", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace("–", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].astype(str)
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(".", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.replace(" ", "")
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.upper()
rutsMetlife["Rut_Beneficiario"] = rutsMetlife["Rut_Beneficiario"].str.strip()

rutsMetlife["Relación"] = rutsMetlife["RELACION"].astype(str)
rutsMetlife["Relación"] = np.where(rutsMetlife["Relación"]== "AS","Titular", "Carga")

rutsMetlife["llave"] = rutsMetlife["Poliza"] + rutsMetlife["Rut_Titular"] + rutsMetlife["Rut_Beneficiario"]


rutsMetlife = rutsMetlife[
    ["Poliza",	"Rut_Titular",	"Rut_Beneficiario",	"Relación","llave",
       
    ]
]
rutsMetlife = rutsMetlife.drop_duplicates(
    [
        "Poliza","Rut_Titular",	"Rut_Beneficiario",	"Relación",
    ],
    keep="last",
)
rutsMetlife.shape

In [ ]:

#ruts_metlife = rutsMetlife
#ruts_metlife.to_excel("F:/martin/MetLife SFTP/ruts_metlife.xlsx", index=False)

In [ ]:
ruts_metlife = pd.concat([ruts_metlife, rutsMetlife], axis=0)
ruts_metlife = ruts_metlife.drop_duplicates(
    [
        "Poliza","Rut_Titular",	"Rut_Beneficiario",	"Relación",
    ],
    keep="last",
)
ruts_metlife.shape

In [ ]:

reportes_metlife = "C:/Users/jmartinez/Documents/Metlife_derty.xlsx"
reportes_metlife_fredy = pd.read_excel(reportes_metlife)

In [ ]:
reportes_metlife_fredy["llave"] =  reportes_metlife_fredy["Rut"]
ruts_metlife["LLave"] = ruts_metlife["Rut_Beneficiario"] 

In [ ]:
reportes_metlife_fredy = pd.merge(
    left=reportes_metlife_fredy,
    right=ruts_metlife,
    how="left",
    left_on="llave",
    right_on="LLave",
)

In [ ]:
reportes_metlife_fredy.head(5)

In [ ]:
#reportes_metlife_fredy.to_excel("C:/Users/jmartinez/Documents/reportes_metlife2.xlsx", index=False)